In [1]:
import torch
import adapters
from adapters import T5AdapterModel
from transformers import T5ForConditionalGeneration, AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig

/mnt/shared_home/vlialin/miniconda3/envs/peft_comparison/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-09-14 18:12:43,914] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")
# model = T5ForConditionalGeneration.from_pretrained("t5-base")
quantization_config = None
# BitsAndBytesConfig(
    # load_in_4bit=True,
    # bnb_4bit_compute_dtype=torch.bfloat16,
    # bnb_4bit_use_double_quant=True,
    # bnb_4bit_quant_type="nf4",
    # load_in_8bit=True,
# )

model = AutoModelForSeq2SeqLM.from_pretrained(
    "t5-base", quantization_config=quantization_config, torch_dtype=torch.bfloat16)
adapters.init(model)
model.add_adapter("adapter", config="pfeiffer", set_active=True)
model.train_adapter("adapter")

/mnt/shared_home/vlialin/miniconda3/envs/peft_comparison/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at t5-base and are newly initialized: ['decoder.embed_tokens.weight', 'encoder.embed_tokens.weight', 'lm_he

RuntimeError: data set to a tensor that requires gradients must be floating point or complex dtype

In [30]:
input_ids = tokenizer("Earth rotates around <extra_id_0>", return_tensors="pt").input_ids
decoder_input_ids = tokenizer("<pad> <extra_id_0>", return_tensors="pt").input_ids

out = model.generate(input_ids=input_ids, decoder_input_ids=decoder_input_ids)
tokenizer.decode(out[0])

'<pad><extra_id_0></s></s>'

In [ ]:
# model = T5AdapterModel.from_pretrained("t5-small")
# model.add_seq2seq_lm_head("lm_head")